I am building my own LLM so first I am building a tokenizer of my own 

STEP 1 - CREATE THE TOKENS

In [2]:
with open("01 Harry Potter and the Sorcerers Stone.txt", "r", encoding="Utf-8") as f:
    raw_text = f.read()

print(len(raw_text)) #Total number of characters
print(raw_text[:99])

439478
M r. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly nor


In [3]:
import re  #library for splitting
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30]) # Remember this is a list having all the tokens

['M', 'r', '.', 'and', 'Mrs', '.', 'Dursley', ',', 'of', 'number', 'four', ',', 'Privet', 'Drive', ',', 'were', 'proud', 'to', 'say', 'that', 'they', 'were', 'perfectly', 'normal', ',', 'thank', 'you', 'very', 'much', '.']


STEP 2 - CREATE TOKEN ID FOR THIS TOKENS

In [4]:
#now we have to use vocabulary that is the sorted words
words = sorted(set(preprocessed))
vocab_size = len(words)
print(vocab_size)

7572


In [5]:
#Remember vocab is a mapping from tokens to tokenIDs
vocab = {token:integer for integer, token in enumerate(words)}

In [6]:
#enumerate - this I have used to assign integer values to the sorted words
for i, item in enumerate(vocab.items()):
    print(item)
    if i>=50:
        break

('!', 0)
("'", 1)
('(', 2)
(')', 3)
('*', 4)
(',', 5)
('-', 6)
('-bodied', 7)
('.', 8)
('1', 9)
('1473', 10)
('1637', 11)
('17', 12)
('1709', 13)
('1945', 14)
('2', 15)
('3', 16)
('31', 17)
('382', 18)
('4', 19)
(':', 20)
(';', 21)
('?', 22)
('A', 23)
('ALBUS', 24)
('ALLEY', 25)
('ALLOWED', 26)
('AM', 27)
('AND', 28)
('ANYTHING', 29)
('ARE', 30)
('AT', 31)
('About', 32)
('According', 33)
('Adalbert', 34)
('Add', 35)
('Adrian', 36)
('Africa', 37)
('African', 38)
('After', 39)
('Against', 40)
('Ages', 41)
('Agrippa', 42)
('Ah', 43)
('Ahead', 44)
('Alberic', 45)
('Albus', 46)
('Albus…”', 47)
('Algie', 48)
('Alicia', 49)
('All', 50)


WE NEED TO REMEMBER TWO CONCEPTS i.e. ENCODER AND DECODER

ENCODER -> it will take text as input and give token ID as output

DECODER -> it will take token ID as input and will give text as output

In [7]:
class SimpleTokenizer:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [8]:
#Now let us create an object of the above class and check if it is returning the ids or not
tokenizer = SimpleTokenizer(vocab)
text =  """It was on the corner of the street that he noticed"""
ids = tokenizer.encode(text)
print(ids)

[593, 6597, 4470, 6131, 2269, 4452, 6131, 5888, 6126, 3488, 4427]


In [9]:
tokenizer.decode(ids)

'It was on the corner of the street that he noticed'

So encoder and decoder is perfectly working as expected

NOW WE HAVE COMPLETED A SIMPLE TOKENIZER OF OUR OWN BUT THERE ARE SOME LIMITATIONS. WE CANNOT USE OTHER TEXT OTHER THAN THE ONE IN THE txt file WE USED MEANING IF I WRITE SOME TEXT THERE WILL BE ERROR SO WE NEED SPECIAL CONTEXT TOKENS WHICH IS ALSO USED BY GPT MODELS .

<|endofText|> and <|unk|> -> unknown will be used

In [10]:
tokens = sorted(list(set(preprocessed)))
tokens.extend(["<|endofText|>", "<|unk|>"])# Here I used extend it will add 2 extra entries see the result it is 7574 but previously it was 7572
vocab = {token:integer for integer, token in enumerate(tokens)}
len(vocab.items())

7574

Let us check 

In [11]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('…Oak', 7569)
('…Then', 7570)
('…”', 7571)
('<|endofText|>', 7572)
('<|unk|>', 7573)


In [12]:
class AdvancedTokenizer:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self,text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

Now let us try again with some random text

In [13]:
tokenizer = AdvancedTokenizer(vocab)

text1 = "Hello, my name is Subhranil Mondal"
text2 = "I am from Kolkata."

text = " <|endofText|> ".join((text1,text2))
print(text)

Hello, my name is Subhranil Mondal <|endofText|> I am from Kolkata.


In [14]:
tokenizer.encode(text)

[7573, 5, 4318, 4332, 3758, 7573, 7573, 7572, 575, 1391, 3164, 7573, 8]

In [15]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, my name is <|unk|> <|unk|> <|endofText|> I am from <|unk|>.'

See this is a word based tokenizer but GPT uses subword tokenizer so for that we need Byte Pair encoder

In [16]:
#we will be using the BPE tokenizer which OpenAI uses
! pip3 install tiktoken


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import importlib
import tiktoken

In [18]:
tokenizer = tiktoken.get_encoding("gpt2")

In [19]:
#Let us test

text = "Hello, my name is Subhranil Mondal. I am from Kolkata."
integers = tokenizer.encode(text, allowed_special={"<|endofText|>"})
print(integers)

[15496, 11, 616, 1438, 318, 3834, 16848, 346, 27328, 282, 13, 314, 716, 422, 509, 13597, 1045, 13]


In [20]:
s = tokenizer.decode(integers)
print(s)

Hello, my name is Subhranil Mondal. I am from Kolkata.


Therefore encoder and decoder is working pretty well

Now, we have to create Input-Target pairs
But what is it?
see we need to do self supervised learning kind of unsupervised. The data are not labeled here . LLM will be feed one word and it will predict next one This is what we gonna implement which the GPT does.

In [21]:
with open("01 Harry Potter and the Sorcerers Stone.txt", "r", encoding="Utf-8") as f:
    raw_text = f.read()

encoded_text = tokenizer.encode(raw_text)
print(len(encoded_text))

124336


In [22]:
sample = encoded_text[50:]

In [23]:
conSize = 4 #context size -> length of input sequence
x = sample[:conSize]
y = sample[1:conSize+1]
print(f"x: {x}")
print(f"y: {y}")

x: [11428, 11, 780, 484]
y: [11, 780, 484, 655]


In [24]:
for i in range(1, conSize+1):
    context = sample[:i]
    desired = sample[i]

    print(context, "->", desired)

[11428] -> 11
[11428, 11] -> 780
[11428, 11, 780] -> 484
[11428, 11, 780, 484] -> 655


Now just decode it

In [25]:
for i in range(1, conSize+1):
    context = sample[:i]
    desired = sample[i]

    print(tokenizer.decode(context), "->", tokenizer.decode([desired]))

 mysterious -> ,
 mysterious, ->  because
 mysterious, because ->  they
 mysterious, because they ->  just


so now we have to turn tokens into embeddings and implement a data loader as inputs and PyTorch Tensors as target like you can think about multidimensional arrays.

STEP 3 - IMPLEMENT A DATALOADER(WE WILL BE USING PYTORCH)

In [26]:
! pip3 install torch torchvision torchaudio


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
import torch
from torch.utils.data import Dataset, DataLoader
class GPTDataset(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        # Step 1 - we will tokenize the entire text
        token_ids = tokenizer.encode(txt,allowed_special={"<|endofText|>"})
        # Step 2 - we will use sliding window to chunk the book into overlapping sequences
        for i in range(0, len(token_ids)-max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    # Step 3 - Then return the total no. of rows
    def __len__(self):
        return len(self.input_ids)
    
    # Step 4 - Return a single row from the datset
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [28]:
# Now we have to create the data loader
def create_dataloader(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDataset(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)
    return dataloader

Now Let us Test

In [29]:
with open("01 Harry Potter and the Sorcerers Stone.txt", "r", encoding="Utf-8") as f:
    raw_text = f.read()

In [30]:
# Now I am gonna convert dataloader into a python iterator to fetch the next entry

import torch
dataloader = create_dataloader(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
iter_data = iter(dataloader)
first_batch = next(iter_data)
print(first_batch)

[tensor([[ 44, 374,  13, 290]]), tensor([[ 374,   13,  290, 9074]])]


see as we used stride=1 so it will overlap so doing this will make the model overfit and noisy .Lets see if we use stride=4 and increase batch_size to 8

In [31]:
dataloader = create_dataloader(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
iter_data = iter(dataloader)
inputs, targets = next(iter_data)
print("\nInputs:\n", inputs)
print("\nTargets:\n", targets)


Inputs:
 tensor([[   44,   374,    13,   290],
        [ 9074,    13,   360,  1834],
        [ 1636,    11,   286,  1271],
        [ 1440,    11,  4389, 16809],
        [ 9974,    11,   547,  6613],
        [  284,   910,   326,   484],
        [  547,  7138,  3487,    11],
        [ 5875,   345,   845,   881]])

Targets:
 tensor([[  374,    13,   290,  9074],
        [   13,   360,  1834,  1636],
        [   11,   286,  1271,  1440],
        [   11,  4389, 16809,  9974],
        [   11,   547,  6613,   284],
        [  910,   326,   484,   547],
        [ 7138,  3487,    11,  5875],
        [  345,   845,   881,    13]])


batch size = 8 means the input tensors are also 8 rows

Now let us create token or vector embeddings

In [32]:
input_ids = torch.tensor([2,3,5,1])

FOR SIMPLICITY WE WILL USE VOCAB SIZE 6

In [33]:
vocab_size = 6
output_dim = 3 # Output dimensions
# Output will be a 6 X 3 matrix
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)
print(embedding_layer(input_ids))

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)
tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


Now see this is just a basic implementation of token embedding but this is not the right way. we meed to have the concept of positional embeddings -> absolute and relative. GPT or any other transfromer uses absolute embeddings

In [34]:
vocab_size = 50257 #Lets give the actual vocab_size for GPT2
output_dim = 256 #Lets keep it 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [35]:
#Now let us create the data loader again
max_length = 4
dataloader = create_dataloader(raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [ ]:
print("\nInput IDs:\n", inputs)
print("\nTarget IDs:\n", targets)
print("\nInputs shape:", inputs.shape) # 8 x 4 tensor


Input IDs:
 tensor([[   44,   374,    13,   290],
        [ 9074,    13,   360,  1834],
        [ 1636,    11,   286,  1271],
        [ 1440,    11,  4389, 16809],
        [ 9974,    11,   547,  6613],
        [  284,   910,   326,   484],
        [  547,  7138,  3487,    11],
        [ 5875,   345,   845,   881]])

Target IDs:
 tensor([[  374,    13,   290,  9074],
        [   13,   360,  1834,  1636],
        [   11,   286,  1271,  1440],
        [   11,  4389, 16809,  9974],
        [   11,   547,  6613,   284],
        [  910,   326,   484,   547],
        [ 7138,  3487,    11,  5875],
        [  345,   845,   881,    13]])

Inputs shape: torch.Size([8, 4])


Each token ID is now embedded as a 256- dimensional vector

Now let us create an embedding layer

In [41]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [ ]:
context_length = max_length
# no.of rows = context_length and no.of columns = output_dim
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [ ]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
#Here torch.arange is creating a sequence of numbers from 0 to max_length-1
print(pos_embeddings.shape)

torch.Size([4, 256])


Now let us add the 4x256 dimensional tensor to each 4x256 token embeddings

In [42]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
